# Equal-Weight S&P 500 Index Fund

## Library Imports

Importing all the libraries which we'll be needing.

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks


In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Acquiring an API Token

We use IEX Cloud API to gather all stock related data which we'll be using.
We store API key in a file named mysecrets.

In [3]:
from mysecrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call





In [4]:
stockSymbol='AAPL'
api_url=f'https://sandbox.iexapis.com/stable/stock/{stockSymbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

We need to parse it and gather only data which we need i.e, Stock price & Market Capitalization.

In [5]:
price = data['latestPrice']
marketCap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame.

In [6]:
my_columns=['Ticker','Stock Price','Market Capitalization']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization


In [7]:
final_dataframe.append(
    pd.Series(
        [
            stockSymbol,
            price,
            marketCap
            
        ],
        index=my_columns
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization
0,AAPL,176,2832369629083


## Looping Through The Tickers in Our List of Stocks

Instead of the above code we can use loop and collect data for all the stocks.

In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stockSymbol in stocks['Ticker'][:50]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stockSymbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([stockSymbol,
                                                   data['latestPrice'], 
                                                   data['marketCap'],
                                                  ],
                                                  index = my_columns), 
                                        ignore_index = True)

In [9]:
final_dataframe

,Ticker,Stock Price,Market Capitalization
0,A,142.44,42946964905
1,AAL,15.22,9924446017
2,AAP,212.41,13161632820
3,AAPL,176.13,2880490066457
4,ABBV,145.45,254452673458
5,ABC,152.76,32132890971
6,ABMD,294.56,13214733421
7,ABT,110.90,197970483992
8,ACN,319.29,218454693824
9,ADBE,449.59,210813028350


## Calculating the Number of Shares to Buy



In [10]:
portfolio_size=input('Enter the size of your portfolio:')

try:
    val=float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")
    val=float(portfolio_size)

Enter the size of your portfolio:1000000


In [11]:
position_size=val/len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Stock Price'][i])
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number Of Shares to Buy
0,A,142.44,42946964905,140.0
1,AAL,15.22,9924446017,1314.0
2,AAP,212.41,13161632820,94.0
3,AAPL,176.13,2880490066457,113.0
4,ABBV,145.45,254452673458,137.0
5,ABC,152.76,32132890971,130.0
6,ABMD,294.56,13214733421,67.0
7,ABT,110.90,197970483992,180.0
8,ACN,319.29,218454693824,62.0
9,ADBE,449.59,210813028350,44.0


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.



### Initializing our XlsxWriter Object

In [12]:
writer = pd.ExcelWriter('trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer , 'Trades', index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [13]:
string_format = writer.book.add_format(
        {
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [14]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Trades'].write(f'{column}1', column_formats[column][0], string_format)
    


## Saving Our Excel Output

Saving our Excel file is very easy:

In [15]:
writer.save()